# SpaceTime Playground

In [1]:
%reload_ext autoreload
%autoreload 3

import torch
import torch.nn.functional as F
from torchinfo import summary
import pytorch_lightning as pl
from src.b2bnet import B2BNetSpaceTimeModel, OtkaDataModule, OtkaTimeDimSplit
from pytorch_lightning.loggers import TensorBoardLogger

In [2]:
# Experiment
segment_size = 120 * 3  # 3sec
batch_size = 256
n_channels = 59
n_features = 32
hidden_size = 16
max_epochs = 200

datamodule = OtkaTimeDimSplit(segment_size=segment_size, batch_size=batch_size)

model = B2BNetSpaceTimeModel(
    n_channels=n_channels, n_features=n_features,
    hidden_size=hidden_size, kernel_size=1, n_subjects=52)

trainer = pl.Trainer(max_epochs=max_epochs,accelerator='cpu', log_every_n_steps=1, deterministic='warn')

trainer.fit(model, datamodule=datamodule)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name        | Type                 | Params
-----------------------------------------------------
0 | autoencoder | SpaceTimeAutoEncoder | 9.3 K 
1 | cls         | Linear               | 34    
-----------------------------------------------------
9.3 K     Trainable params
0         Non-trainable params
9.3 K     Total params
0.037     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


## spatioTemporal test
this code is used to first encode the spatial characteristics of EEG data (positional information) and then encode the temporal characteristics of EEG data (time information). The output of this code is a 3D matrix of size (number of channels, number of time points, number of spatial bins). This code is used to generate the data for the spatioTemporal decoding.

In [1]:
import torch
from torch import nn

In [19]:
# DATA
import numpy as np
X = torch.rand(5, 1000, 30)
# zero padding along the time domension
X = torch.nn.functional.pad(X, pad=(0, 0, 64, 0), mode='constant', value=0)
del X

torch.Size([5, 100, 10, 30])


In [3]:
# generate random DATA
input = torch.randn(1, 120, 59).permute(0, 2, 1)  # torch.Size([256, 360, 59])

# model
# encoder
m = nn.Sequential(
    nn.Conv1d(59, 30, 1, stride=1),
    nn.ReLU(),
    nn.Conv1d(30, 15, 1, stride=1),
    nn.ReLU(),
)
output = m(input)

m = nn.LSTM(15, 10, batch_first=True)  # num_layers can be detemined by hyperparameter search
output = m(output.permute(0, 2, 1))

# decoder
m = nn.LSTM(10, 15, batch_first=True)
output = m(output[0])
m = nn.Sequential(
    nn.ConvTranspose1d(15, 30, 1, stride=1),
    nn.ReLU(),
    nn.ConvTranspose1d(30, 59, 1, stride=1),
)
output = m(output[0].permute(0, 2, 1))

output.shape

torch.Size([1, 59, 120])